# Double and Dueling Deep Q Networks (DDQNs)

Now that we've taken a look at [Deep Q Networks](dqn.ipynb), we'll examine some of their shortcomings. By tacking on some extra algorithmic elements to the DQN training loop that alleviate these issues, we can improve the stability and performance of our Q-learners.

Do you remember how we update our Deep Q network from the last notebook? 
$$Q^{\pi}(s_t, a_t) \leftarrow \left(r_{t+1} + \gamma \, \underset{a \in A}{\operatorname{max}} \space Q^{\pi}(s_{t+1}, a) \right)$$

Let's first designate the target we're trying to update $Q^{\pi}(s_t, a_t)$ to as the **Temporal Difference Target**, or **TD target** for short: 

$$ TD_{target} = \left(r_{t+1} + \gamma \, \underset{a \in A}{\operatorname{max}} \space Q^{\pi}(s_{t+1}, a) \right)$$

Another way to think about this update is that we're trying to minimize the **Temporal Difference error**:
$$TD_{error} = \left[r_{t+1} + \gamma \, \underset{a \in A}{\operatorname{max}} \space Q^{\pi}(s_{t+1}, a) - Q^{\pi}(s_t, a_t) \right]^2$$
$$\Delta_w\ Q^{\pi}(s_t, a_t)= - \alpha\ \nabla_w \left[TD_{error}\right] Q^{\pi}(s_t, a_t)$$
$$ Q^{\pi}(s_t, a_t) \leftarrow Q^{\pi}(s_t, a_t) + \Delta_w\ Q^{\pi}(s_t, a_t)$$

The $\alpha$ above represents the learning rate for the Q-network, and $\nabla_w \left[TD_{error}\right] Q^{\pi}(s_t, a_t)$ represents the gradient of the weights in $Q^{\pi}(s_t, a_t)$ with respect to $TD_{error}$.


### What's the problem with this?

### Problem #1:

The first is that we're kind of chasing our own tail. If we look at the equation for our TD target, we're actually shifting our target every time we update $Q^{\pi}(s_{t}, a_t)$.
$$ TD_{target} = \left(r_{t+1} + \gamma \, \underset{a \in A}{\operatorname{max}} \space Q^{\pi}(s_{t+1}, a) \right)$$

Remember that the On-Policy Action-Value Function, $Q^{\pi}(s,a)$, gives the expected return if you start in state s, take an arbitrary action a, and then *forever after act according to policy $\pi$*. 

We can imagine us being a novice chess player: every time we improve our ability to evaluate the board, positions with tricky tactics that we used to fumble now give us new opportunities for attacks. This means that positions that were low-value to our past self are now high-value. 

In practice, this appears as training instability. If the learning rate is high, the Q-network has a tendency to oscillate between different strategies as the Q-value targets rapidly shift. This heavily constrains the effective learning rates for basic Q-networks and reduces the effective training speed and performance of such learners.

This issue is known as the **moving-target problem**.

### Problem #2:
Let's imagine that we're in a very simple 1-dimensional linear world made of a few blocks, $L_1, L_2... L_{100}$, A, S, B, and R.
$L_1, L_2... L_{100}$ as well as R are terminal states. The states in each block is reachable from a neighboring block. For example, from the starting position, the agent can choose to move to state A, and then choose to move to any of the states in $L_1, L_2... L_{100}$. 

Every time a state in $L_1, L_2... L_{100}$ is reached, the agent receives a reward sampled from a normal distribution with mean -0.1 and variance 1. If the agent reaches R, the reward is a constant 0.

| L_1, L_2,... L_100 | A | S | B | R |
|--------------------|---|---|---|---|

Given the description above, we'd expect the learned behavior to be to always go right to B, since the expected reward is 0 instead of -0.1. However, consider that our TD target is the reward that is obtained under the *best-case* action:
 $$ TD_{target} = \left(r_{t+1} + \gamma \, \underset{a \in A}{\operatorname{max}} \space Q^{\pi}(s_{t+1}, a) \right)$$
 It's extremely likely that our learner will continuously update to go left, as the best action will almost certainly have a reward greater than 0. This is known as **maximization-bias**. More colloquially, we know this as the tendency for DQNs to be overconfident about the quality of states with high variance, which leads to risk-taking behavior.

---
Both Double DQNs and Dueling DQNs are optimizations aimed to solve the moving-target problem and maximization bias. The mechanisms for the two are different and can be implemented simultaneously.

Fun fact: The world of value-based learning used to largely revolve around stacking these "hotfixes" for DQN problems. One seminal paper by OpenAI achieved superhuman performance (on average) across 57 Atari games by combining *6* of these optimizations!

<img src="Rainbow_DQN.png" alt="Rainbow DQN performance" width="500"/>

## Double DQNs

Let's examine the core crux of the moving-target problem:
$$Q^{\pi}(s_t, a_t) \leftarrow \left(r_{t+1} + \gamma \, \underset{a \in A}{\operatorname{max}} \space Q^{\pi}(s_{t+1}, a) \right)$$

The Q network is used twice in the training loop: once to guide the policy in gathering experience, and once again in the TD update rule to estimate the value of the next state, given that we take the action we think is best: $\underset{a \in A}{\operatorname{argmax}} \space Q^{\pi}(s_{t+1}, a)$.

Let's make a frozen copy of the Q-network that we call the **target network**, which we'll denote as $Q_t^{\pi}(s_t, a_t)$. The target network, being frozen, will not receive gradient updates. We'll offload the next-state value estimation to the target network, while we'll keep the training Q-network to guide the policy: 

$$Q^{\pi}(s_t, a_t) \leftarrow \left[r_{t+1} + \gamma \, Q_t^{\pi}\left(s_t, \underset{a \in A}{\operatorname{argmax}} \space Q^{\pi}(s_{t+1}, a) \right)\right]$$
$$\pi(s) = \begin{cases}
\underset{a \in A}{\operatorname{argmax}} \space Q(s, a) & \text{with probability } 1 - \epsilon \\
\text{random action} & \text{with probability } \epsilon
\end{cases}$$

Now, our target is constant. Of course, we have to update our target network somehow, since we do want the model to ultimately learn the correct value function. There are two common methods for doing so:

1. The hard update: Every *update_target* steps, we copy all of the weights of the Q-network to the target network. 
$$ Q_t^{\pi}(s_t, a_t) \leftarrow  Q^{\pi}(s_t, a_t)$$
This has the property of maintaining a constant target and updating it in large jumps.

2. The soft update: We set a hyperparameter $\tau$, and update the target network as follows every step:
$$ Q_t^{\pi}(s_t, a_t) \leftarrow \tau\ Q^{\pi}(s_t, a_t) + (1-\tau)\ Q_t^{\pi}(s_t, a_t)$$
This causes the target network to lag behind the target network and constrains the target network values to change slowly, which also improves stability. In the [DDPG paper](https://arxiv.org/pdf/1509.02971.pdf), the soft update was empirically tested to be better than the hard update.

<details><summary>Why does Double DQN alleviate maximization-bias?</summary>

$$Q^{\pi}(s_t, a_t) \leftarrow \left[r_{t+1} + \gamma \, Q_t^{\pi}\left(s_t, \underset{a \in A}{\operatorname{argmax}} \space Q^{\pi}(s_{t+1}, a) \right)\right]$$

The network that gathers experience is different from the network that estimates the target of the next move. Previously, an overconfident deep-Q network would consistently sample a state and update on it. Now that the policy-determining network is, to some degree, independent from the estimation-calculating network, it's less likely that both learners will be overconfident in the same way and cause the same fixation as in naive DQNs. Of course, this doesn't solve the issue. A much better solution comes in the form of Dueling DQNs, which we'll explore below.

</details>

### Sidenote: Don't be confused by common misconceptions!

While conducting research for this tutorial, we found a few articles online that have a slightly different Double-DQN update rule to the update rule described in the [DDQN paper.](https://arxiv.org/abs/1509.06461):

$$Q^{\pi}(s_t, a_t) \leftarrow \left[r_{t+1} + \gamma \, Q^{\pi}\left(s_t, \underset{a \in A}{\operatorname{argmax}} \space Q_t^{\pi}(s_{t+1}, a) \right)\right]$$

Note that they flip the position of the primary and target Q-networks so that the primary Q-network becomes the value estimator and the target Q-network informs the policy (by deciding which move in the next step is the best move). This negates a major advantage of Double DQNs - namely, that the fast-learning primary network can explore different policies while the lagging-behind target network provides a stable move evaluation. 

## Dueling DQNs

Recall the on-policy quality and value functions from the first DQN notebook:
$$V^{\pi}(s) = \underset{\tau \sim \pi}{\operatorname{E}}[{R(\tau)\left| s_0 = s\right.}]$$
$$Q^{\pi}(s,a) = \underset{\tau \sim \pi}{\operatorname{E}}[{R(\tau)\left| s_0 = s, a_0 = a\right.}]$$

We can separate $Q^{\pi}(s,a)$ into an **Advantage** aspect and a **value** aspect:
$$Q^{\pi}(s,a) = A^{\pi}(s, a) + V^{\pi}(s)$$

Since the Q-value represents the quality of a state-action pair and the V-value represents the expected value of a given state, the **advantage** is the difference between the two and represents the advantage that a given action has relative to the status quo.

Our Q-value estimator implicitly calculates both the value of each state as well as the advantage of each move relative to the others; the Dueling DQN paper shows empirically that performance benefits can be gained by explicitly separating the estimation of Value and Advantage into two separate heads.

#### One network, two outputs
We can construct a Dueling DQN's Advantage and Value functions with two heads that share a majority of initial layers. The advantage of this (pun not intended) is that key features are helpful in calculating both Advantage and Value. Since we share the first layers between the two heads, we don't have to calculate these key features twice.

<img src="Dueling_DQN.png" alt="Dueling DQN architecture" width="500"/>

Recall that we choose our actions as follows:
$$\pi(s) = \begin{cases}
\underset{a \in A}{\operatorname{argmax}} \space Q(s, a) & \text{with probability } 1 - \epsilon \\
\text{random action} & \text{with probability } \epsilon
\end{cases}$$

To find the policy, we still need to reconstruct the Q-value of each move.

### Reconstructing Q

Given that we've set Advantage to be the difference between Quality and Value, it might be intuitive to set $$Q^{\pi}(s,a)\ \text{to}\ A^{\pi}(s, a) + V^{\pi}(s)$$
However, recall the definitions of $V^{\pi}(s)$, $Q^{\pi}(s,a)$, and $\pi(s)$:

$$V^{\pi}(s) = \underset{\tau \sim \pi}{\operatorname{E}}[{R(\tau)\left| s_0 = s\right.}]$$
$$Q^{\pi}(s,a) = \underset{\tau \sim \pi}{\operatorname{E}}[{R(\tau)\left| s_0 = s, a_0 = a\right.}]$$
$$\pi(s) = \begin{cases}
\underset{a \in A}{\operatorname{argmax}} \space Q(s, a) & \text{with probability } 1 - \epsilon \\
\text{random action} & \text{with probability } \epsilon
\end{cases}$$

The best case scenario for $V^{\pi}(s)$, which happens when $\epsilon = 0$, is defined as:$$\underset{a \in A}{\operatorname{max}}\ Q^{\pi}(s,a)$$
This is because a policy that always takes the optimal action under $Q^{\pi}(s,a)$ will set $V^{\pi}(s)$ to the quality of the best action at state $s$.

Unfortunately, since $A^{\pi}(s, a) + V^{\pi}(s)$ are neural function estimators, this property isn't preserved: it's possible for $$V^{\pi}(s) > \left[\underset{a \in A}{\operatorname{max}}\ Q^{\pi}(s,a) = \underset{a \in A}{\operatorname{max}}\ A^{\pi}(s,a) + V^{\pi}(s) \right]$$

This happens when $\underset{a \in A}{\operatorname{max}}\ A^{\pi}(s,a) < 0$. As random initialiations often violate this property, the network experiences a number of training instabilities. We alleviate this by anchoring $V^{\pi}(s)$ to $\underset{a \in A}{\operatorname{max}}\ Q^{\pi}(s,a)$ by setting Q as follows:

$$Q^{\pi}(s,a) = V^{\pi}(s) + \left[A^{\pi}(s, a) - \underset{a \in A}{\operatorname{max}} \space A_t^{\pi}(s, a)\right]$$

Note that the maximum value V can take on in the above equation is when $a =  \underset{a \in A}{\operatorname{max}} \space A_t^{\pi}(s, a)$, where the expression within the brackets evaluates to 0 and $V^{\pi}(s) = Q^{\pi}(s,a)$.

### Dueling Update Rule
Now that we have the Q value, the training of the Dueling DQN is identical to that of the standard DQN. The update function distributes the update between the advantage and the value.

If the current step isn't the last step:
$$Q^{\pi}(s_t, a_t) \leftarrow \left(r_{t+1} + \gamma \, \underset{a \in A}{\operatorname{max}} \space Q^{\pi}(s_{t+1}, a) \right)$$
$$V^{\pi}(s) + \left[A^{\pi}(s, a) - \underset{a \in A}{\operatorname{max}} \space A_t^{\pi}(s, a)\right] \leftarrow \left(r_{t+1} + \gamma \, \underset{a \in A}{\operatorname{max}} \space \left[V^{\pi}(s) + A^{\pi}(s, a) - \underset{a \in A}{\operatorname{max}} \space A_t^{\pi}(s, a)\right] \right)$$

If the current step is the last step:
$$Q^{\pi}(s_t, a_t) \leftarrow r_{t+1}$$

$$V^{\pi}(s) + \left[A^{\pi}(s, a) - \underset{a \in A}{\operatorname{max}} \space A_t^{\pi}(s, a)\right] \leftarrow r_{t+1}$$